# Step1: Unzipping File and Showing Statistics

In [ ]:
import shutil
shutil.unpack_archive('../input/melanoma-partial-augmentation-for-balance-512x512/Melanoma-JPEG-512.zip', './')


import os
N_data = 53

path, dirs, files = next(os.walk("./validation/benign"))
print('Benign samples in validation set:',len(files))

path, dirs, files = next(os.walk("./validation/malignant"))
print('Malignant samples in validation set:',len(files))

benign_training_samples = []
malignant_training_samples = []

for iter_data in range(N_data):
    path = './tr' + str(iter_data) + '/benign'
    path, dirs, files = next(os.walk(path))
    benign_training_samples.append(len(files))
    #print('Benign samples in training set'+str(iter_data)+':',len(files))
    
    path = './tr' + str(iter_data) + '/malignant'
    path, dirs, files = next(os.walk(path))
    malignant_training_samples.append(len(files))
    #print('Malignant samples in training set'+str(iter_data)+':',len(files))

print('Benign samples in training set:',benign_training_samples)
print('Malignant samples in training set:',malignant_training_samples)

# Step2: DataLoader

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy




plt.ion()   # interactive mode

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    0: transforms.Compose([
        transforms.Resize(512),
        transforms.Resize((560,560)),
        transforms.RandomRotation(15,),
        transforms.RandomResizedCrop(512),
        transforms.RandomGrayscale(p=0.1),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomPerspective(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276])
    ]),
    1: transforms.Compose([
        transforms.Resize(512),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276])
    ]),
}

data_dir = './'
data_folders = []

for iter_data in range(N_data):
    data_folders.append('./tr' + str(iter_data))
data_folders.append('validation')
    
print(data_folders)

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x == 'validation'])
                  for x in data_folders}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=14,
                                             shuffle=True, num_workers=0)
              for x in data_folders}
dataset_sizes = {x: len(image_datasets[x]) for x in data_folders}
class_names = image_datasets['validation'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    #plt.savefig('test.pdf', dpi = 300)
    plt.pause(0.001)  # pause a bit so that plots are updated

# Step3: Observing Augmentation for Training and Validation Data 

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders[data_folders[10]]))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out)#title=[class_names[x] for x in classes])

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['validation'])) #53rd element in data_folders

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out)# title=[class_names[x] for x in classes])

# Step4: Declaring Neural Networks

#### Declaring several NNs. However, we are using one of the combinations. We may use another combination for investigating a different pre-trained NN or for investigating a different FC.

In [ ]:
#model_name = 'vgg19_bn'
model_ft = models.vgg19_bn(pretrained=True)
num_ftrs = model_ft.classifier[0].in_features
layer_width = 512 #Small for Resnet, large for VGG


#model_ft = models.wide_resnet101_2(pretrained=True)
#num_ftrs = model_ft.fc.in_features
#layer_width = 20 #Small for Resnet, large for VGG


half_in_size = round(num_ftrs/2)

Num_class=2

class SpinalNet(nn.Module):
    def __init__(self):
        super(SpinalNet, self).__init__()
        
        self.fc_spinal_layer1 = nn.Sequential(
            nn.Linear(half_in_size, layer_width),
            nn.ReLU(inplace=True),)
        self.fc_spinal_layer2 = nn.Sequential(
            nn.Linear(half_in_size+layer_width, layer_width),
            nn.ReLU(inplace=True),)
        self.fc_spinal_layer3 = nn.Sequential(
            nn.Linear(half_in_size+layer_width, layer_width),
            nn.ReLU(inplace=True),)
        self.fc_spinal_layer4 = nn.Sequential(
            nn.Linear(half_in_size+layer_width, layer_width),
            nn.ReLU(inplace=True),)
        self.fc_out = nn.Sequential(
            nn.Linear(layer_width*4, Num_class),)
        
    def forward(self, x):
        x1 = self.fc_spinal_layer1(x[:, 0:half_in_size])
        x2 = self.fc_spinal_layer2(torch.cat([ x[:,half_in_size:2*half_in_size], x1], dim=1))
        x3 = self.fc_spinal_layer3(torch.cat([ x[:,0:half_in_size], x2], dim=1))
        x4 = self.fc_spinal_layer4(torch.cat([ x[:,half_in_size:2*half_in_size], x3], dim=1))
        
        x = torch.cat([x1, x2], dim=1)
        x = torch.cat([x, x3], dim=1)
        x = torch.cat([x, x4], dim=1)
  
        x = self.fc_out(x)
        return x
        
VGG_fc = nn.Sequential(
            nn.Linear(num_ftrs, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, Num_class)
        )


'''
Changing the fully connected layer to SpinalNet or VGG or ResNet
'''

#model_ft.fc = nn.Linear(num_ftrs, 2) # SpinalNet() # 
model_ft.classifier = SpinalNet() # VGG_fc #
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()

# Step5: The Training Function

In [ ]:
import csv


def train_model_imbalanced (model, criterion, optimizer, scheduler, phases_data):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    optimal_acc = 0.0
    optimal_F1 = 0.0
    
    file = open('train_details.csv', 'a+', newline ='')
    with file:     
        write = csv.writer(file) 

        for phase in phases_data:
            if phase != 'validation':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            malignant_phase = 0
            malignant_correctly_predicted = 0
            malignant_predicted = 0


            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase != 'validation'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase != 'validation':
                        loss.backward()
                        optimizer.step()
                    if phase == 'validation':
                        for iter_l in range(len(labels)):
                            if labels[iter_l] == 1: # Malignant
                                malignant_phase = malignant_phase + 1
                                if preds[iter_l] == 1: 
                                    malignant_correctly_predicted = malignant_correctly_predicted + 1
                            if preds[iter_l] == 1: 
                                malignant_predicted = malignant_predicted + 1


                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            output_format = [phase, epoch_loss, epoch_acc]
            write.writerows([output_format]) 

            # deep copy the model
            if phase == 'validation': 
                precision = malignant_correctly_predicted/malignant_predicted
                recall = malignant_correctly_predicted/malignant_phase
                F1_score = 2/(1/precision+1/recall)
                if F1_score + epoch_acc < optimal_F1 + optimal_acc: #considering both factors equally
                    print('F1 score: {:.4f}'.format(F1_score))
                    model.load_state_dict(best_model_wts) # if no improvement, start from previous best model
                    continue
                optimal_F1 = F1_score
                optimal_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                time_elapsed = time.time() - since
                print('Time from Start {:.0f}m {:.0f}s, F1 score: {:.4f}'.format(
                    time_elapsed // 60, time_elapsed % 60, F1_score))
            #print()

        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
        print('Optimal val Acc: {:4f}'.format(optimal_acc))

        # load best model weights

        model.load_state_dict(best_model_wts)
    return model

# Step6: Training

In [ ]:
phases = []
interval_val = 5 

for iter_data in range(54):
    phases.append('./tr' + str(iter_data%N_data)) # calling all traiing folders
    if (iter_data%interval_val) == (interval_val-1) or iter_data > 47:
        phases.append('validation') # calling the validation data at a certain interval
                                    # Also, calling frequently, near the end of training
print('Phases:', phases)

In [ ]:
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft = train_model_imbalanced(model_ft, criterion, optimizer_ft, exp_lr_scheduler, phases)


# Step7: Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sn
import pandas as pd

y_pred = []
y_true = []

# iterate over test data
for inputs, labels in dataloaders['validation']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        output = model_ft(inputs) # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output) # Save Prediction
        
        labels = labels.data.cpu().numpy()
        y_true.extend(labels) # Save Truth

# constant for classes
classes = ('benign', 'malignant')

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix, index = [i for i in classes],
                     columns = [i for i in classes])



plt.figure(figsize = (4,2),dpi=100)
plt.rcParams['font.size'] = '16'
sn.heatmap(df_cm, annot=True, fmt=".0f")

print(classification_report(y_true, y_pred, target_names = ['benign', 'malignant']))

# Step8: Saving Model

In [ ]:
best_net = model_ft.classifier

PATH = "./best_model.pt"
torch.save(best_net.state_dict(), PATH)


'''
#Guideline for loading model in future 

import sys
sys.modules[__name__].__dict__.clear()

device = torch.device('cpu')

model_ft = models.wide_resnet101_2(pretrained=True)
model_fc = nn.Linear(num_ftrs, 2)

model_fc.load_state_dict(torch.load(PATH, map_location=device))
model_ft.fc = model_fc

'''

In [ ]:
# Deleting Unzipped files for smaller output size
import shutil
for name_folders in data_folders:
    shutil.rmtree(name_folders)

#plt.savefig('output.pdf')